In [ ]:
import numpy as np
from frontend import App

app = App.create("domino")
V, F, T = app.mesh.box().tetrahedralize().scale(0.1, 0.3, 0.025)
app.asset.add.tet("block", V, F, T)
scene = app.scene.create("domino")

R, minR, d, N, C = 1.25, 0.6, 0.15, 4096, 3
xz = None
for i in reversed(range(N)):
    t = 2.0 * C * np.pi * i / N - np.pi / 2
    r = (R - minR) * i / N + minR
    angle = 180 * t / np.pi
    x, z = -r * np.cos(t), r * np.sin(t)
    if xz is None or np.linalg.norm(np.array([x, z]) - xz) > d:
        scene.add("block").at(x, 0, z).rotate(angle, "y")
        xz = np.array([x, z])

scene.add("block").at(-0.15, 0.25, -R).rotate(90, "y").rotate(-20, "z")

gap = app.session.param().get("contact-ghat")
scene.add.invisible.wall([0, scene.min("y") - 0.5 * gap, 0], [0, 1, 0])

fixed = scene.build().report().shading(shading={"flat": True})
fixed.preview()

In [ ]:
param = app.session.param()
(
    param.set("volume-young-mod", 6000)
    .set("volume-poiss-rat", 0.49)
    .set("friction", 0.1)
    .set("min-newton-steps", 32)
    .set("dt", 0.01)
    .set("fps", 15)
    .set("frames", 180)
)

session = app.session.create(fixed)
session.start(param).preview()
session.stream()

In [ ]:
session.animate()

In [ ]:
# this is for CI
assert session.finished()

In [ ]:
if not app.CI:
    # An extremely stiff case with a quadratic barrier
    extreme_param = app.session.param()
    (
        extreme_param.set("volume-young-mod", 1e5)
        .set("volume-poiss-rat", 0.49)
        .set("friction", 0.1)
        .set("frames", 32)
        .set("min-newton-steps", 32)
        .set("dt", 0.01)
        .set("fps", 15)
        .set("barrier", "quad")
    )
    quad_session = app.session.create(fixed)
    quad_session.start(extreme_param).preview()
    quad_session.stream()

In [ ]:
if not app.CI:
    quad_session.animate()

In [ ]:
if not app.CI:
    # The same extremely stiff case with our cubic barrier
    extreme_param.set("barrier", "cubic")
    cubic_session = app.session.create(fixed)
    cubic_session.start(extreme_param).preview()
    cubic_session.stream()

In [ ]:
if not app.CI:
    cubic_session.animate()